# Introduction by Example

> [PyTorch geometric](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html#learning-methods-on-graphs)의 튜토리얼을 구현한 jupyter notebook 입니다.

![](./images/pytorch_geometric.png)

## 01. Common Benchmark Datasets

PyTorch Geometric contains a large number of common benchmark datasets, *e.g.* all Planetoid datasets (Cora, Citeseer, Pubmed), all graph classification datasets from http://graphkernels.cs.tu-dortmund.de/ and their [cleaned versions](https://github.com/nd7141/graph_datasets), the QM7 and QM9 dataset, and a handful of 3D mesh/point cloud datasets like FAUST, ModelNet10/40 and ShapeNet.

### Cora Dataset

This time, the `Data` objects holds a label for each node, and additional attributes: `train_mask`, `val_mask` and `test_mask`:

- `train_mask` denotes against which nodes to train (140 nodes)
- `val_mask` denotes which nodes to use for validation, *e.g.*, to perform early stopping (500 nodes)
- `test_mask` denotes against which nodes to test (1000 nodes)

In [2]:
from torch_geometric.datasets import Planetoid


dataset = Planetoid('../data/Cora', name='Cora')

Processing...
Done!


In [3]:
dataset

Cora()

In [4]:
len(dataset)

1

In [5]:
dataset.num_classes

7

In [6]:
dataset.num_node_features

1433

In [7]:
data = dataset[0]
data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [12]:
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils.convert import to_networkx


G = to_networkx(data, to_undirected=True)
# nx.draw(G)

In [13]:
data.is_undirected()

True

In [15]:
data.train_mask.sum().item()

140

In [16]:
data.val_mask.sum().item()

500

In [17]:
data.test_mask.sum().item()

1000

## 02. Mini-batches